In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
df = pd.read_csv('data/merged_data_brand_dropna.csv')

In [3]:
# datetime 컬럼 변환
df['event_time_moscow'] = pd.to_datetime(df['event_time_moscow'])

In [4]:
import pandas as pd

# view와 purchase만 필터링
df_filtered = df[df['event_type'].isin(['view', 'purchase'])].copy()

# 시간 정렬
df_filtered['event_time_moscow'] = pd.to_datetime(df_filtered['event_time_moscow'])
df_filtered.sort_values(by=['user_id', 'product_id', 'event_time_moscow'], inplace=True)

# 각 user_id + product_id 조합 내에서 view 후 purchase 시 차이 계산
df_filtered['time_diff'] = df_filtered.groupby(['user_id', 'product_id'])['event_time_moscow'].diff()

# purchase에서만 차이 계산 (view → purchase)
df_purchase = df_filtered[df_filtered['event_type'] == 'purchase'].copy()

# product_id별 평균 소요시간 계산 (초 단위 → 분으로 변환)
product_avg_time = df_purchase.groupby('product_id')['time_diff'].mean().reset_index()
product_avg_time['time_diff_minutes'] = product_avg_time['time_diff'].dt.total_seconds() / 60

# 소요시간이 짧은 Top 10 제품 추출
top10_fastest = product_avg_time.sort_values(by='time_diff_minutes').head(10)

# 상품명 등 추가 정보 합치고 싶으면 아래 merge 예시 사용
# top10_fastest = top10_fastest.merge(df[['product_id', 'category_id', 'brand']].drop_duplicates(), on='product_id', how='left')

# 컬럼 정리
top10_fastest = top10_fastest[['product_id', 'time_diff_minutes']]

# CSV 저장
top10_fastest.to_csv("product_top10_fastest_to_purchase.csv", index=False)

print(top10_fastest)


       product_id  time_diff_minutes
19060     5886525           0.000000
16812     5868351           0.516667
7263      5800320           0.533333
22578     5921619           0.550000
9063      5812623           0.583333
17924     5877707           0.616667
15385     5859459           0.683333
15706     5862230           0.700000
1474      5652258           0.733333
8740      5811697           0.816667


- 구매까지 평균 소요시간 안 구하는 걸로

### 조회 비율 높은 제품

In [15]:
import pandas as pd

# 1. 전체 유저 수
total_users = df['user_id'].nunique()

# 2. 제품별 조회 유저 수
product_view_users = df[df['event_type'] == 'view'].groupby('product_id')['user_id'].nunique().reset_index()
product_view_users.columns = ['product_id', 'view_user_count']

# 3. 유저 수 대비 조회 비율 계산
product_view_users['view_ratio'] = product_view_users['view_user_count'] / total_users * 100  # 퍼센트로 보기 좋게

# 4. 상위 제품 정렬
top_products = product_view_users.sort_values(by='view_ratio', ascending=False)

In [18]:
# 결과 확인
print(top_products.head(20))

       product_id  view_user_count  view_ratio
12681     5809910            53367    4.433893
12683     5809912            17780    1.477217
7922      5751383            15937    1.324094
24023     5877454            15716    1.305733
24025     5877456            15143    1.258127
7961      5751422            14787    1.228549
2938      5649236            13615    1.131176
18733     5849033            13326    1.107165
20256     5856186            13209    1.097444
12682     5809911            12325    1.023999
12348     5808664            10655    0.885250
14244     5816170            10307    0.856337
8679      5769877            10105    0.839554
18254     5846437             9536    0.792280
3687      5677043             8883    0.738027
18964     5850281             8549    0.710277
26235     5896420             8467    0.703464
5550      5700037             8196    0.680949
1952      5560754             8169    0.678705
10214     5785144             6945    0.577012


In [20]:
# CSV로 저장
top_products.head(10).to_csv('data/06_top_10_products_by_view_ratio.csv', index=False)
top_products.head(20).to_csv('data/06_top_20_products_by_view_ratio.csv', index=False)